###  DEFINITIONS


In [30]:
# Necessary Imports
from flask import Flask, jsonify
from werkzeug.routing import IntegerConverter
from _csv import Dialect as _Dialect
import csv
import copy
import os
from rtls_slave import aoa_main, pixel_calculate
import json

# PARAMETER FOR COMPRESSING THE PIXEL POINTS
BETA = 1
BIAS = 0

# Handle Negative Values
class SignedIntConverter(IntegerConverter):
    regex = r'-?\d+'
app = Flask(__name__)
app.url_map.converters['signed_int'] = SignedIntConverter
#Store input values: AoA and Horizontal Angle

aoabefore = {}
aoa= {}
lev = []

@app.route('/')
def home():
    return "This is the HomePage"

#Routing Settings
@app.route('/send/<signed_int:horiz>', methods=['GET'])
def get_angle(horiz):
    global aoa
    global aoabefore
    print("Plane Angle: " + str(horiz))
    aoa_result = aoa_main()
    print(aoa_result)
    input_letter = input("If you want to add this data to list type 'y', otherwise press Enter")
    if input_letter!='y':
        return jsonify({'state':'Try Again'})
    lev.append(horiz)
    aoabefore = copy.deepcopy(aoa)
    aoa = aoa_result

    with open(f"./aoa_results/two_down_indoor_{horiz}.json",'w') as f:
        json.dump(aoa_result, f)
        
    return jsonify({'state': 'success'})

@app.route('/calculate/', methods=['GET'])
def calculate():
    print("Before Turning："+str(aoabefore)+'\n'+"After Turning： " + str(aoa) +'\n'+"Rotation Angle： "+ str(lev))
    #aoa_image()
    u = []
    v = []

    set_aoa = set(aoa)
    set_aoabefore = set(aoabefore)

    for slave in set_aoa.intersection(set_aoabefore):
        minnumber = min(len(aoabefore[slave]), len(aoa[slave]))
        f = open(f"./pixels/{slave.replace(':','_')}_before.csv",'a',newline="")
        f.truncate()
        writer = csv.writer(f)
        
        for i in range(2,minnumber-2):
            for j in range(2,minnumber-2):
                (temp1, temp2) = pixel_calculate(lev[-2] - lev[-1], aoabefore[slave][i], aoa[slave][j],beta = BETA,aoa_bias=BIAS)

                if (temp1 != 1):
                    u.append(temp1)
                    v.append(temp2)
                    tupx = (temp1,temp2)
                    writer.writerow(tupx)
                
        
        f.close()

    
    return jsonify({'state': 'success'})

def post_calculation(level1, level2):

    #  AoA BEFORE
    with open(f'./aoa_results/aoa_{level1}.json', 'r') as f:
        aoabefore = json.load(f)
    
    # AoA
    with open(f'./aoa_results/aoa_{level2}.json','r') as f:
        aoa = json.load(f)

    lev = [level1, level2]
    
    print("Before Turning："+str(aoabefore)+'\n'+"After Turning： " + str(aoa) +'\n'+"Rotation Angle： "+ str(lev))
    #aoa_image()
    u = []
    v = []

    set_aoa = set(aoa)
    set_aoabefore = set(aoabefore)

    for slave in set_aoa.intersection(set_aoabefore):
        minnumber = min(len(aoabefore[slave]), len(aoa[slave]))
        f = open(f"./pixels/{slave.replace(':','_')}_before.csv",'a',newline="")
        f.truncate()
        writer = csv.writer(f)
        
        for i in range(2,minnumber-2):
            for j in range(2,minnumber-2):
                (temp1, temp2) = pixel_calculate(lev[-2] - lev[-1], aoabefore[slave][i], aoa[slave][j],beta = BETA,aoa_bias=BIAS)

                if (temp1 != 1):
                    u.append(temp1)
                    v.append(temp2)
                    tupx = (temp1,temp2)
                    writer.writerow(tupx)
                
        
        f.close()

### RUNNING THE SEVER

In [31]:
app.run(host='0.0.0.0',port = 5000, debug=False)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://10.19.217.212:5000
Press CTRL+C to quit


Plane Angle: 65
Master : <RTLSNode(CC26x2 Master, started daemon 21276)> 
Passives : [] 
All : [<RTLSNode(CC26x2 Master, started daemon 21276)>]
Devices Reset
Start scan for 5 sec
Scan Results: [{'addr': '80:6F:B0:EE:9C:67', 'addrType': 0, 'rssi': -66, 'advSID': 255, 'periodicAdvInt': 0}, {'addr': '80:6F:B0:EE:AA:13', 'addrType': 0, 'rssi': -69, 'advSID': 1, 'periodicAdvInt': 240}, {'addr': '80:6F:B0:EE:AA:13', 'addrType': 0, 'rssi': -61, 'advSID': 255, 'periodicAdvInt': 0}]
Try to connect to : 80:6F:B0:EE:AA:13
Connected to : 80:6F:B0:EE:AA:13 with connection handle 0
AOA Params Set for Connection Handle: 0
AOA Callback Set for All Connection Handlers
AOA Started with Connection Handle: 0
Going to sleep for 10 sec
Try to stop AOA result parsing threadSTOP Command Received

AOA Stopped
Master Disconnected
Done
{'80:6F:B0:EE:AA:13': [10, 13, 11, 10, 10, 13, 8, 9, 10, 9, 7, 7, 8, 7, 10, 10, 8, 11, 12, 12, 10, 12, 13, 10]}


10.19.190.195 - - [27/Mar/2023 19:08:37] "GET /send/65 HTTP/1.1" 200 -


Plane Angle: 65
Master : <RTLSNode(CC26x2 Master, started daemon 15640)> 
Passives : [] 
All : [<RTLSNode(CC26x2 Master, started daemon 15640)>]
Devices Reset
Start scan for 5 sec
Scan Results: [{'addr': '80:6F:B0:EE:AA:13', 'addrType': 0, 'rssi': -68, 'advSID': 255, 'periodicAdvInt': 0}, {'addr': '80:6F:B0:EE:9C:67', 'addrType': 0, 'rssi': -66, 'advSID': 255, 'periodicAdvInt': 0}, {'addr': '80:6F:B0:EE:9C:67', 'addrType': 0, 'rssi': -58, 'advSID': 1, 'periodicAdvInt': 240}]
Try to connect to : 80:6F:B0:EE:9C:67
Connected to : 80:6F:B0:EE:9C:67 with connection handle 0
AOA Params Set for Connection Handle: 0
AOA Callback Set for All Connection Handlers
AOA Started with Connection Handle: 0
Going to sleep for 10 sec
Try to stop AOA result parsing threadSTOP Command Received

AOA Stopped
Master Disconnected
Done
{'80:6F:B0:EE:9C:67': [0, 3, -1, -4, -6, -6, -8, -8, -9, -12, -12, -8, -12, -5, -1, 8, 15, 16, 19, 15, 14, 12, 5]}


...

10.19.190.195 - - [27/Mar/2023 19:13:45] "GET /send/71 HTTP/1.1" 200 -


Plane Angle: 73
Master : <RTLSNode(CC26x2 Master, started daemon 15124)> 
Passives : [] 
All : [<RTLSNode(CC26x2 Master, started daemon 15124)>]
Devices Reset
Start scan for 5 sec
Scan Results: [{'addr': '80:6F:B0:EE:9C:67', 'addrType': 0, 'rssi': -60, 'advSID': 255, 'periodicAdvInt': 0}, {'addr': '80:6F:B0:EE:AA:13', 'addrType': 0, 'rssi': -66, 'advSID': 255, 'periodicAdvInt': 0}, {'addr': '80:6F:B0:EE:AA:13', 'addrType': 0, 'rssi': -60, 'advSID': 1, 'periodicAdvInt': 240}, {'addr': '80:6F:B0:EE:9C:67', 'addrType': 0, 'rssi': -57, 'advSID': 1, 'periodicAdvInt': 240}]
Try to connect to : 80:6F:B0:EE:AA:13
Connected to : 80:6F:B0:EE:AA:13 with connection handle 0
Try to connect to : 80:6F:B0:EE:9C:67
Connected to : 80:6F:B0:EE:9C:67 with connection handle 1
AOA Params Set for Connection Handle: 0
AOA Params Set for Connection Handle: 1
AOA Callback Set for All Connection Handlers
AOA Started with Connection Handle: 0
AOA Started with Connection Handle: 1
Going to sleep for 10 sec
Try to

10.19.190.195 - - [27/Mar/2023 19:15:39] "GET /send/73 HTTP/1.1" 200 -


Plane Angle: 75
Master : <RTLSNode(CC26x2 Master, started daemon 2668)> 
Passives : [] 
All : [<RTLSNode(CC26x2 Master, started daemon 2668)>]
Devices Reset
Start scan for 5 sec
Scan Results: [{'addr': '80:6F:B0:EE:AA:13', 'addrType': 0, 'rssi': -72, 'advSID': 1, 'periodicAdvInt': 240}, {'addr': '80:6F:B0:EE:9C:67', 'addrType': 0, 'rssi': -68, 'advSID': 1, 'periodicAdvInt': 240}, {'addr': '80:6F:B0:EE:AA:13', 'addrType': 0, 'rssi': -63, 'advSID': 255, 'periodicAdvInt': 0}, {'addr': '80:6F:B0:EE:9C:67', 'addrType': 0, 'rssi': -59, 'advSID': 255, 'periodicAdvInt': 0}]
Try to connect to : 80:6F:B0:EE:AA:13
Connected to : 80:6F:B0:EE:AA:13 with connection handle 0
Try to connect to : 80:6F:B0:EE:9C:67
Connected to : 80:6F:B0:EE:9C:67 with connection handle 1
AOA Params Set for Connection Handle: 0
AOA Params Set for Connection Handle: 1
AOA Callback Set for All Connection Handlers
AOA Started with Connection Handle: 0
AOA Started with Connection Handle: 1
Going to sleep for 10 sec
Try to s

10.19.190.195 - - [27/Mar/2023 19:17:01] "GET /send/75 HTTP/1.1" 200 -


### POST CALCULATION

In [ ]:
# POST-ANALYSIS
BIAS = -50
BETA = .6
post_calculation(69,75)

### ANGLE CALCULATION

In [ ]:
# ELEVATION AND AZIMUTH ANGLE VERIFICATION
from rtls_slave import pixel_calculate,angle_calculate
import csv

def calculate():
    print("Before Turning："+str(aoabefore)+'\n'+"After Turning： " + str(aoa) +'\n'+"Rotation Angle： "+ str(lev))
    #aoa_image()
    angles = {}

    for i in range(len(aoabefore)):
        for j in range(len(aoa)):
            (temp1, temp2) = pixel_calculate(lev[-2] - lev[-1], aoabefore[i], aoa[j])
            angles[(aoabefore[i],aoa[j])] = (temp1,temp2)
    return angles

aoabefore = [47, 48, 47, 48, 49, 49, 47, 49, 49, 47, 48, 48]
aoa = [48, 48, 44, 45, 43, 43, 44, 43, 47, 45, 46]
aoabefore = [i-45 for i in aoabefore ]
aoa = [i-45 for i in aoa]
lev = [74,77]
comp_parameter = 1
print(calculate())